### Part2

In [1]:
!pip3 install pyspark
!pip3 install findspark

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: my awesome Spark program")
	.master("spark://172.31.2.53:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

# You can read the data from a file into DataFrames
#df = spark.read.csv("hdfs://172.31.2.53:9000/export.csv")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 03:03:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = (spark.read
         .format("csv")
         .option("inferSchema", True)
         .option("header", True)
         .load("hdfs://172.31.2.53:9000/export.csv"))
print(df.show(10))

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [5]:
sorted_df = df.orderBy(['cca2', 'timestamp'], ascending=[True, True])

In [6]:
print(sorted_df.show(10))

+-------------+---------+----+----+--------------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|                  cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+--------------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            5|     1217|  AE| ARE|United Arab Emirates|      501|device-mac-501e4O...|      48|  213.42.16.154|    24.0|yellow|     54.0|Celsius|  16|1458444054343|
|            0|      915|  AR| ARG|           Argentina|      227|meter-gauge-2273p...|      34|  200.71.230.81|   -34.6| green|   -58.38|Celsius|  15|1458444054251|
|            1|     1189|  AR| ARG|           Argentina|      319|meter-gauge-319Y3...|      54| 200.71.236.145|   -34.6|yellow|   -58.38|Celsius|  25|1458444054287|
|   

In [7]:
sorted_df.write.mode("overwrite").csv("hdfs://172.31.2.53:9000/sorted_export.csv", header=True)

In [12]:
spark.stop()

### Part3

In [2]:
"""
This is a skeleton of the PageRank algorithm.
Feel free to use any piece of code in this provided skeleton source file.
To use it, you will need to copy it into your Notebook. 
Feel free to make modifications to template code as you see fit.
However, you are encouraged to implement the algorithm completely on
your own. :-)
"""
import re
import sys
from operator import add
from typing import Iterable, Tuple

from pyspark.resultiterable import ResultIterable
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql import SparkSession

In [4]:
"""Helper function to calculates URL contributions to the rank of other URLs"""
def calculateRankContrib(urls: ResultIterable[str], rank: float) -> Iterable[Tuple[str, float]]:
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


"""Helper function to parses a urls string into urls pair"""
def parseNeighborURLs(urls: str) -> Tuple[str, str]:
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [5]:
# Initialize the Spark context
# TODO: You should define a new name for your PySpark PageRank program
spark = (SparkSession.builder.appName("A2:PageRank")
    .master("spark://172.31.2.53:7077")
    .config("spark.executor.memory", "2048M")
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 04:32:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
"""Hint: You want to use hash partitioning to optimize the performance of join"""

# Loads in input file
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...
linesRDD = spark.sparkContext.textFile("hdfs://172.31.2.53:9000/web-BerkStan.txt")

# Perform a transformation to define a links RDD by using parseNeighborURLs helper function
linksRDD = linesRDD.map(lambda urls: parseNeighborURLs(urls)).distinct().groupByKey()

# Initialize a ranks RDD
ranksRDD = linksRDD.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [7]:
# Calculates and updates URL ranks continuously using PageRank algorithm.
    # Replace N with a number ranging from 3 to 10
    # For A2 you are required to complete 10 iterations\n",
for iteration in range(10):
    contributeRDD = linksRDD.join(ranksRDD).flatMap(
        lambda url_urls_rank: calculateRankContrib(url_urls_rank[1][0], url_urls_rank[1][1]) 
    )
    ranksRDD = contributeRDD.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank)

### Task1

In [8]:
highestRanks = ranksRDD.sortBy(lambda x: x[1], ascending=False)
highest50 = highestRanks.toDF(["url","rank"])
highest50.show(50)

+------+------------------+
|   url|              rank|
+------+------------------+
|272919|6531.3246237524545|
|438238| 4335.323158564439|
|571448|2383.8976074118877|
|601656|2195.3940755967287|
|316792|1855.6908757901508|
|319209|1632.8193684975686|
|184094|1532.2842374483398|
|571447|1492.9301630938783|
|401873|1436.1600933469292|
| 66244|1261.5783958673344|
| 68949|1260.7919421349136|
|284306|1257.2475650644851|
| 68948|1251.1723536459228|
| 77284|1235.2985405976258|
| 68946|1235.2985405976258|
| 95552|1235.2985405976256|
| 96070|1235.2985405976256|
| 86238|1235.2985405976256|
| 86239|1235.2985405976256|
| 86237|1235.2985405976256|
| 95551|1235.2985405976256|
| 68947|1235.2985405976256|
| 66909|1235.2985405976256|
|   768|1225.5975665113078|
|   927|1117.8383051141839|
|210376| 920.6701252803681|
| 95527|   919.67971465211|
|100130| 916.0190658202696|
|101163| 912.5380530105955|
| 95018| 911.1831080077997|
|100646| 909.7095673033023|
| 96045| 904.3981315809757|
| 66879|  895.790974

In [10]:
# Save output to HDFS
ranksDf = ranksRDD.toDF()
# Note: You should not overwrite the input file. Otherwise you need to reload it
ranksDf.write.format("csv").save("hdfs://172.31.2.53:9000/web-BerkStan_output_1.csv")

### Task2

In [14]:
from pyspark import StorageLevel
linksRDD_part = linksRDD.partitionBy(8)
ranksRDD_part = linksRDD_part.map(lambda url_neighbors: (url_neighbors[0], 1.0)).partitionBy(8).persist()

In [15]:
for iteration in range(10):
    contributeRDD_part = linksRDD_part.join(ranksRDD_part).flatMap(
        lambda url_urls_rank: calculateRankContrib(url_urls_rank[1][0], url_urls_rank[1][1])
    )
    if iteration % 3 == 0:
        ranksRDD_part = contributeRDD_part.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank).persist()
    else:
        ranksRDD_part = contributeRDD_part.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank)

In [16]:
top50_part = ranksRDD_part.sortBy(lambda x: x[1], ascending=False)
topDF = top50_part.toDF(["url","rank"])
topDF.show(50)

+------+------------------+
|   url|              rank|
+------+------------------+
|272919| 6531.324623752437|
|438238| 4335.323158564453|
|571448| 2383.897607411885|
|601656|2195.3940755967305|
|316792| 1855.690875790141|
|319209|1632.8193684975693|
|184094| 1532.284237448335|
|571447|1492.9301630938762|
|401873| 1436.160093346927|
| 66244|1261.5783958673348|
| 68949|1260.7919421349154|
|284306| 1257.247565064484|
| 68948|1251.1723536459235|
| 77284|1235.2985405976272|
| 95551|1235.2985405976272|
| 68946| 1235.298540597627|
| 86237| 1235.298540597627|
| 95552| 1235.298540597627|
| 86239| 1235.298540597627|
| 68947|1235.2985405976265|
| 96070| 1235.298540597626|
| 66909| 1235.298540597626|
| 86238|1235.2985405976256|
|   768|1225.5975665113028|
|   927|1117.8383051141818|
|210376| 920.6701252803678|
| 95527| 919.6797146521109|
|100130| 916.0190658202705|
|101163| 912.5380530105966|
| 95018| 911.1831080078007|
|100646| 909.7095673033034|
| 96045| 904.3981315809767|
| 66879|  895.790974

### Task3 - kill a worker and then run task 2 again